<a href="https://colab.research.google.com/github/zi2p/M-machine-learning/blob/main/M33001_%D0%9F%D0%B8%D1%81%D0%B0%D1%80%D0%B5%D0%B2%D0%B0_lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Деревья решений

## Построение дерева

Опишем жадный алгоритм построения бинарного дерева решений:
1. Начинаем со всей обучающей выборки $X$, которую помещаем в корень $R_1$. 
2. Задаём функционал качества $Q(X, j, t)$ и критерий остановки. 
3. Запускаем построение из корня: $SplitNode(1, R_1)$

Функция $SplitNode(m, R_m)$
1. Если выполнен критерий остановки, то выход.
2. Находим наилучший с точки зрения $Q$ предикат: $j, t$: $[x_j<t]$
3. Помещаем предикат в вкршину и получаем с его помощью разбиение $X$ на две части: $R_{left} = \lbrace x|x_j<t \rbrace$ и $R_{right} = \lbrace x|x_j \geqslant t \rbrace$
4. Поместим $R_{left}$ и $R_{right}$ соответсвенно в левое и правое поддерево.
5. Рекурсивно повторяем $SplitNode(left, R_{left})$ и $SplitNode(right, R_{right})$.

В конце поставим в соответствие каждому листу ответ. Для задачи классификации - это самый частый среди объектов класс или вектор с долями классов (можно интерпретировать как вероятности):
$$ c_v = \arg \max_{k\in Y} \sum_{(x_i,y_i) \in R_v} [y_i=k]  $$

## Функционал качества для деревьев решений


Энтропия Шеннона для системы с N возможными состояниями определяется по формуле:
$$H = - \sum_{i=0}^{N} p_i\log_2p_i $$

где $p_i$ – вероятности нахождения системы в $i$-ом состоянии. 

Это очень важное понятие теории информации, которое позволяет оценить количество информации (степень хаоса в системе). Чем выше энтропия, тем менее упорядочена система и наоборот. С помощью энтропии мы формализуем функционал качества для разделение выборки (для задачи классификации).

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn import datasets
import random
from pprint import pprint
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

Код для расчёта энтропии:

In [ ]:
def entropy(y):
    _, counts = np.unique(y, return_counts=True)
    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
    return entropy

Здесь $y$ - это массив значений целевой переменной

Энтропия – по сути степень хаоса (или неопределенности) в системе. Уменьшение энтропии называют приростом информации (information gain, IG).

Обочначим $R_v$ - объекты, которые нужно разделить в помощью предиката в вершине $v$. Запишем формулу для расчёта информационного прироста:
$$ Q = IG = H(R_v) - (H(R_{left})+H(R_{right}))$$

На каждом шаге нам нужно максимизировать этот функционал качества. Как это делать? Например, так можно перебрать $t$ для выбранного $j$.

Предыдущая версия формулы прироста информации слишком упрощена. В работе необходимо использовать более устойчивую формулу, которая учитывает не только энтропию подмножеств, но и их размер. 

$$ Q = IG = H(R_v) - \Big (\frac{|R_{left}|} {|R_{v}|} H(R_{left})+ \frac{|R_{right}|} {|R_{v}|} H(R_{right})\Big)$$

где, $|R_{v}|$, $|R_{left}|$ и $|R_{right}|$ - количество элементов в соответствующих множествах.


### Задание 4.1

Реализуйте алгоритм построения дерева. Должны быть отдельные функции (методы):
* для расчёта энтропии (уже есть), 
* для разделения дерева (используйте `pandas`), 
* для подсчёта функционала качества $IG$, 
* для выбора наилучшего разделения (с учетом признаков и порогов),
* для проверки критерия остановки.

Для набора данных `iris` реализуйте алгоритм и минимум три из разными критерия остановки из перечисленных ниже:
* максимальной глубины дерева `max_depth` = 5  
* минимального числа объектов в листе = 5
* максимальное количество листьев в дереве = 5
* purity (остановка, если все объекты в листе относятся к одному классу)

Реализуйте функцию `predict` (на вход функции подаётся датафрейм с объектами)

Оцените точность каждой модели с помощью метрики доля правильных ответов (`from sklearn.metrics import accuracy_score` или реализовать свою).

In [ ]:
def do_split(df, index, limit):
    left = df[df[df.columns[index]] < limit]
    right = df[df[df.columns[index]] >= limit]
    return left, right

def count_IG(df, part_left, part_righr):
    size_v = len(df[df.columns[-1]])
    size_left = len(part_left[part_left.columns[-1]])
    size_right = len(part_righr[part_righr.columns[-1]])
    IG = entropy(df[df.columns[-1]]) - (size_left / size_v * entropy(part_left[part_left.columns[-1]]) + size_right / size_v * entropy(part_righr[part_righr.columns[-1]]))
    return IG

def get_best_split(df):
    best_index = 0
    best_limit = 0.
    best_IG = 0.
    for index in range(len(df.columns) - 1):
        for lim in df[df.columns[index]]:
            left, right = do_split(df, index, lim)
            cur_IG = count_IG(df, left, right)
            if cur_IG > best_IG:
                best_index, best_limit, best_IG = index, lim, cur_IG
    return {'index': best_index, 'value': best_limit, 'children': do_split(df, best_index, best_limit)}

In [ ]:
def split_node(root, max_depth=5, min_size=5, purity=0, depth=1):
    left, right = root['children']
    del (root['children'])

    # проверяем на принадлежность к одному классу, чтобы выйти
    if purity:
        if left.value_counts().size == 1:
            c_v(left)
            return
        if right.value_counts().size == 1:
            c_v(right)
            return

    # проверяем на пустоту        
    if left.empty or right.empty:
        root['left'] = c_v(pd.concat([left, right]))
        root['right'] = root['left']
        return

    # выходим, если достигли наибольшей глубины    
    if depth >= max_depth:
        root['left'] = c_v(left)
        root['right'] = c_v(right)
        return

    # при положении, когда слева осталось много элементов, то продолжаем деление 
    # если осталось минимальное количество, то выходим   
    if len(left) <= min_size:
        root['left'] = c_v(left)
        return
    else:
        root['left'] = get_best_split(left)
        split_node(root['left'], max_depth, min_size, purity, depth + 1)

    # при положении, когда справа осталось много элементов, то продолжаем деление 
    # если осталось минимальное количество, то выходим     
    if len(right) <= min_size:
        root['right'] = c_v(right)
        return
    else:
        root['right'] = get_best_split(right)
        split_node(root['right'], max_depth, min_size, purity, depth + 1)

In [ ]:
def c_v(part):
    return part[part.columns[-1]].value_counts().index[0]

In [ ]:
def build_tree(df, max_depth, min_size, purity):
    root = get_best_split(df)
    split_node(root, max_depth, min_size, purity, 1)
    return root

def predict_line(node, part_df):
    if part_df[part_df.columns[node['index']]].iloc[0] < node['value']:
        if isinstance(node['left'], dict):
            return predict_line(node['left'], part_df)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict_line(node['right'], part_df)
        else:
            return node['right']

def predict_df(root, df_test):
    y_pred = []
    for i in range(df_test.shape[0]):
        part_df = df_test.iloc[[i]]
        pred = predict_line(root, part_df)
        y_pred.append(pred)
    return y_pred

In [ ]:
iris = datasets.load_iris()
df = pd.DataFrame(data=np.c_[iris['data'],iris['target']], columns=iris['feature_names'] + ['target'])

Проверим работу алгоритма. Для этого подготовим данные.

In [ ]:
X_iris = df.iloc[:, :-1]
y_iris = df.iloc[:, -1]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_iris, y_iris)

In [ ]:
X_train.head(10)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
2,4.7,3.2,1.3,0.2
32,5.2,4.1,1.5,0.1
144,6.7,3.3,5.7,2.5
129,7.2,3.0,5.8,1.6
31,5.4,3.4,1.5,0.4
18,5.7,3.8,1.7,0.3
52,6.9,3.1,4.9,1.5
5,5.4,3.9,1.7,0.4
19,5.1,3.8,1.5,0.3
107,7.3,2.9,6.3,1.8


In [ ]:
X_test.head(10)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
102,7.1,3.0,5.9,2.1
53,5.5,2.3,4.0,1.3
104,6.5,3.0,5.8,2.2
91,6.1,3.0,4.6,1.4
47,4.6,3.2,1.4,0.2
20,5.4,3.4,1.7,0.2
86,6.7,3.1,4.7,1.5
113,5.7,2.5,5.0,2.0
83,6.0,2.7,5.1,1.6
21,5.1,3.7,1.5,0.4


In [ ]:
accuracy_scores = []
tree_1 = build_tree(X_train, 5, None, False)
tree_2 = build_tree(X_train, None, 5, False)
tree_3 = build_tree(X_train, None, None, True)
print("tree by max_depth=5:", accuracy_score(y_test, predict_df(tree_1, X_test)))
print("tree by min_size=5:", accuracy_score(y_test, predict_df(tree_2, X_test)))
print("tree by purity:", accuracy_score(y_test, predict_df(tree_3, X_test)))

tree by max_depth=5: 0.9587354954
tree by min_size=5: 0.9535274395
tree by purity: 0.96001642534


С помощью моей реализации нашли оценки качества моделей построенных по разным параметрам. Они получились приблизительно одинаковы.

##  Случайный лес

Опишем алгоритм случайный лес (*random forest*) и попутно разберём основные идеи:

1. Зададим $N$ - число деревьев в лесу.
2. Для каждого $n$ из $N$ сгенерируем свою выборку $X_n$. Пусть $m$ - это количество объектов в $X$. При генерации каждой $X_n$ мы будем брать объекты $m$ раз с возвращением. То есть один и тот же объект может попасть в выборку несколько раз, а какие-то объекты не попадут. (Этот способ назвается бутстрап).
3. По каждой $X_n$ построим решающее дерево $b_n$. Обычно стараются делать глубокие деревья. В качестве критериев остановки можно использовать `max_depth` или `min_samples_leaf` (например, пока в каждом листе не окажется по одному объекту). При каждом разбиении сначала выбирается $k$ (эвристика $k = \sqrt d$, где $d$ - это число признаков объектов из выборки $X$) случайных признаков из исходных, и оптимальное разделение выборки ищется только среди них. Обратите внимание, что мы не выбрасываем оставшиеся признаки!
4. Итоговый алгоритм будет представлять собой результат голосования (для классификации) и среднее арифметическое (для регрессии). Модификация алгоритма предполагает учёт весов каждого отдельного слабого алгоритма в ансамбле, но в этом особо нет смысла.


### Задание 4.2

В качестве набора данных используйте: https://www.kaggle.com/mathchi/churn-for-bank-customers

Там есть описание и примеры работы с этими данными. Если кратко, речь идёт про задачу прогнозирования оттока клиентов. Есть данные о 10 тысячах клиентов банка, часть из которых больше не являются клиентами.

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
churn=pd.read_csv("gdrive/MyDrive/churn.csv", sep = ',')

In [ ]:
churn["Gender"] = churn["Gender"].map({'Female': 1, 'Male': 0})
churn["Geography"] = churn["Geography"].map({'France': 1, 'Spain': 0, 'Germany' : 2})

In [ ]:
churn=churn.drop('Surname', axis=1)

In [ ]:
X = churn.iloc[:, :-1]
y = churn.iloc[:, -1]

In [ ]:
X_train_churn, X_test_churn, y_train_churn, y_test_churn = train_test_split(X, y)

Используя либо свою реализацию, либо  `DecisionTreeClassifier` с разными настройками из `sklearn.tree` реализйте алгоритм "случайный лес". 

Найдите наилучшие гиперпараметры этого алгоритма: 
* количество деревьев, 
* критерий остановки, 
* функционал качества, 
* минимальное количество объектов в листьях 
* другие.

Нельзя использовать готовую реализацию случайного леса из `sklearn`.

В подобных задачах очень важна интерпретируемость алгоритма. Попытайтесь оценить информативность признаков, т.е. ответить а вопрос, значения каких признаков являются самыми важными индикаторами того, что банк потеряет клиента.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
class myRandomForest:
  def new(self, counts_trees=100, max_depth=5, min_samples_leaf=5):
    self.counts_trees = counts_trees
    self.max_depth = max_depth
    self.min_samples_leaf = min_samples_leaf
    self.trees = []

  def fit_myRF(self, X, y):
    features = int(np.sqrt(X.shape[1]))
    for i in range (0, self.counts_trees):
      seed = np.random.randint(1, 1000)
      X_ = X.sample(n=len(X), random_state=seed)
      y_ = y.sample(n=len(X), random_state=seed)
      tree = DecisionTreeClassifier(max_features=features, max_depth=self.max_depth, min_samples_split=self.min_samples_leaf)
      tree.fit(X_, y_)
      self.trees.append(tree)

  def predict_RF(self, X):
    res = []
    for tree in self.trees:
      res.append(tree.predict(X))
    res = pd.DataFrame(data=res)
    return round(res.mean())

  def importance_(self):
    res = []
    for tree in self.trees:
      res.append(tree.feature_importances_)
    res = pd.DataFrame(data = res)
    return res.mean()

In [ ]:
myRF=myRandomForest()
myRF.new()
myRF.fit_myRF(X_train_churn, y_train_churn)

In [ ]:
y_pred = myRF.predict_RF(X_test_churn)
accuracy_score(y_test_churn, y_pred)  

0.8492

Напишем функцию, ишущую лучшие гиперпараметры.


In [ ]:
def get_best_parameters():
  best_score = 0
  best_max_tree = 0
  best_max_depth = 0 
  best_min_samples_leaf = 0 
  for max_tree in [100, 200, 300, 400, 500]:
    for max_depth in range(1, 10):
      for min_samples_leaf in range(2, 5):
        myForest = myRandomForest()
        myForest.new(max_tree, max_depth, min_samples_leaf)
        myForest.fit_myRF(X_train_churn, y_train_churn)
        y_pred = myForest.predict_RF(X_test_churn)
        current_score = accuracy_score(y_test_churn, y_pred)
        if current_score > best_score:
          best_score = current_score
          best_max_tree = max_tree
          best_max_depth = max_depth 
          best_min_samples_leaf = min_samples_leaf
  return best_max_tree, best_max_depth, best_min_samples_leaf

In [ ]:
best_max_tree, best_max_depth, best_min_samples_leaf = get_best_parameters()

Теперь проверим насколько хороши параметры.

In [ ]:
myForest = myRandomForest()
myForest.new(best_max_tree, best_max_depth, best_min_samples_leaf)
myForest.fit_myRF(X_train_churn, y_train_churn)

y_pred = myForest.predict_RF(X_test_churn)
accuracy_score(y_test_churn, y_pred) 

0.8608

In [ ]:
pd.DataFrame(data=np.c_[X_test_churn.columns, myForest.importance_()], columns=["Название признака","Важность"]).sort_values(by="Важность", ascending=False)

,Название признака,Важность
5,Age,0.327468
8,NumOfProducts,0.275865
7,Balance,0.081662
10,IsActiveMember,0.075678
3,Geography,0.058806
2,CreditScore,0.039608
1,CustomerId,0.035047
11,EstimatedSalary,0.034424
0,RowNumber,0.030537
4,Gender,0.019934


Сравним со встроенной Sklearn.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
sklearn_RF=RandomForestClassifier(best_max_tree)
sklearn_RF.fit(X_train_churn, y_train_churn)
y_pred = sklearn_RF.predict(X_test_churn)
accuracy_score(y_test_churn, y_pred) 

0.8608

In [ ]:
pd.DataFrame(data = np.c_[X_test_churn.columns, sklearn_RF.feature_importances_], columns=["Название признака","Важность"]).sort_values(by="Важность", ascending=False) 

,Название признака,Важность
5,Age,0.203963
8,NumOfProducts,0.126251
7,Balance,0.109955
11,EstimatedSalary,0.099953
0,RowNumber,0.099777
1,CustomerId,0.098741
2,CreditScore,0.098614
6,Tenure,0.058387
3,Geography,0.038016
10,IsActiveMember,0.035891


Получили модели, первые три признака которых совпадают. Дальше признаки, найденные моей моделью и моделью SKlearn разнятся. Значит моя модель составлена неочень хорошо.

Лучшие гипер параметры найдены:

* количество деревьев = 200
* максимальной глубины дерева = 9
* минимальное число объектов в листе = 3

In [ ]:
best_max_tree, best_max_depth, best_min_samples_leaf

(200, 9, 3)

Как мне кажется, параметры найдены не лучшем образом. При этом ошибку выдает не сильно плохую (сравнимую с ошибкой sklearn).